In [429]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.image as img
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

### Считываем файл в DataFrame

In [430]:
df = pd.read_csv("cars_moldova_no_dup-_2_.csv")
df

,Make,Model,Year,Style,Distance,Engine_capacity(cm3),Fuel_type,Transmission,Price(euro)
0,Toyota,Prius,2011,Hatchback,195000,1800,Hybrid,Automatic,7750
1,Renault,Grand Scenic,2014,Universal,135000,1500,Diesel,Manual,8550
2,Volkswagen,Golf,1998,Hatchback,1,1400,Petrol,Manual,2200
3,Renault,Laguna,2012,Universal,110000,1500,Diesel,Manual,6550
4,Opel,Astra,2006,Universal,200000,1600,Metan/Propan,Manual,4100
...,...,...,...,...,...,...,...,...,...
37259,Land Rover,Freelander,2002,Crossover,225000,1800,Metan/Propan,Manual,4400
37260,Dacia,Logan Mcv,2015,Universal,89000,1500,Diesel,Manual,7000
37261,Renault,Modus,2009,Hatchback,225,1500,Diesel,Manual,4500
37262,Mazda,6,2006,Combi,370000,2000,Diesel,Manual,4000


### 4.1 Агрегация

#### Статистика цены для разного вида Fuel_type и Transmission

In [431]:
# Сгруппируем Fuel_type и Price(euro)
df1 = df.groupby("Fuel_type")["Price(euro)"].describe()
df1

,count,mean,std,min,25%,50%,75%,max
Fuel_type,,,,,,,,
Diesel,17198.0,10110.951215,10315.742145,1.0,4200.00,7000.0,12500.00,200000.0
Electric,282.0,16665.308511,13215.360244,600.0,8299.25,12999.0,21374.25,130000.0
Hybrid,1976.0,14508.467611,9251.545430,555.0,8860.00,12200.0,18000.00,176000.0
Metan/Propan,4169.0,5342.005757,6377.267393,10.0,2400.00,3800.0,6750.00,200000.0
Petrol,13175.0,8994.996053,87801.589768,5.0,2450.00,5200.0,10000.00,10000000.0
Plug-in Hybrid,464.0,18442.394397,10736.850936,1300.0,12242.50,14000.0,19825.00,63900.0


In [432]:
# Статистика цены для разных типов трансмиссии 
df.groupby(["Transmission"])["Price(euro)"].agg([('median',np.median), ('std',np.std), ('Q5%', lambda x: np.percentile(x, q=5)), ('Q95%',lambda x: np.percentile(x, q=95))] )

,median,std,Q5%,Q95%
Transmission,,,,
Automatic,11600.0,13319.189646,3000.0,36500.0
Manual,4200.0,69575.339491,900.0,11990.0


Так, видно, что автомобиль с автоматической коробкой передач стоит в разы дороже, по сравнению с автомобилем с механической коробкой передач

In [433]:
# Статистика цены для разных типов топливных систем
df.groupby(["Fuel_type"])["Price(euro)"].agg([('median',np.median), ('std',np.std), ('Q5%', lambda x: np.percentile(x, q=5)), ('Q95%',lambda x: np.percentile(x, q=95))] )

,median,std,Q5%,Q95%
Fuel_type,,,,
Diesel,7000.0,10315.742145,1800.00,28414.00
Electric,12999.0,13215.360244,5752.50,39900.00
Hybrid,12200.0,9251.545430,5499.00,29992.25
Metan/Propan,3800.0,6377.267393,1000.00,13500.00
Petrol,5200.0,87801.589768,850.00,24500.00
Plug-in Hybrid,14000.0,10736.850936,8799.15,40924.15


Из полученных результатов видно, что наиболее дорогие автомобили имеют следующий тип топливных систем: "Electric", "Hybrid", "Plug-in Hybrid", автомобили со средней стоимостью: "Diesel", "Petrol",  а дешевые автомобили имеют тип: "Metan/Propan"


### 4.2 Оценка встречаемости разных типов трансмиссий для разных Марок автомобилей (Make) и Моделей автомобилей

In [434]:
# Выполняется группировка по столбцу Year и с помощью метода .count() вычисляется встречаемость разного типа трансмиссии
# для разных типов марок и моделей автомобилей
dff = df.groupby(["Make", "Model", "Transmission"])["Year"].count().reset_index().rename(columns={"Year":"Count"})
dff

,Make,Model,Transmission,Count
0,ARO,Altele,Automatic,1
1,ARO,Altele,Manual,1
2,Abarth,500,Automatic,1
3,Acura,ILX,Automatic,2
4,Acura,MDX,Automatic,4
...,...,...,...,...
1310,Xpeng,G3,Automatic,1
1311,Zaz,1102 Tavria,Manual,5
1312,Zaz,968,Manual,13
1313,Zaz,Altele,Manual,2


Из полученных данных можно увидеть встречаемость того или иного типа трансмиссии для разных Марок/Моделей автомобиля.

Так, например, для марки автомобилей Zaz чаще встречается механическая коробка передач, а для Acura чаще встречается автоматическая коробка передач.

### 4.3 Реализация «умной» замены на редкие категории

In [435]:
# Группировка столбцов по цене 
df_car = df.groupby("Make")["Price(euro)"].mean()
df_car = pd.DataFrame(df_car)
df_car

,Price(euro)
Make,
ARO,7000.000000
Abarth,7200.000000
Acura,10919.857143
Alfa Romeo,6975.000000
Alta marca,4816.592593
...,...
Volkswagen,6660.608652
Volvo,15098.020440
Xpeng,26000.000000


In [436]:
df_car.describe()

,Price(euro)
count,87.000000
mean,21537.262353
std,70111.114895
min,1366.500000
25%,4458.221085
50%,7649.666667
75%,13999.875000
max,627021.750000


In [437]:
# Пусть средняя цена является порогом, т.е ниже этого значения дешевые авто, а выше - дорогие
mean_price = int(df_car.mean())

In [438]:
# Выделяем дешевые машины (цена меньше средней)
df_car1 = df_car[(df_car["Price(euro)"] < mean_price)]
mask1 = df_car["Price(euro)"] < mean_price

In [439]:
df1 = df["Make"].value_counts()
df_cheap = df1[mask1]
df_cheap.tail(50)

Suzuki              176
Subaru              171
Rover               115
Chrysler             97
Jeep                 94
GAZ                  82
Daewoo               79
Mini                 76
Infiniti             72
Dodge                66
Lincoln              57
Moskvich / Izh       51
Alfa Romeo           46
Smart                42
UAZ                  41
Ssangyong            34
Daihatsu             31
Cadillac             27
Alta marca           27
Great Wall           21
Zaz                  20
Chery                19
Lancia               18
Byd                  17
Saab                 15
Lada                 14
Acura                14
Lifan                12
Isuzu                11
LuAZ                 10
Ravon                 5
Buick                 5
Changan               4
Datsun                3
Geely                 3
Haima                 3
Scion                 3
Haval                 3
BAIC                  3
Jac                   3
Pontiac               2
Faw             

In [440]:
df_cheap.describe()

count      74.000000
mean      494.648649
std       924.953378
min         1.000000
25%         5.000000
50%        44.000000
75%       521.500000
max      3953.000000
Name: Make, dtype: float64

In [441]:
# выделим редкие дешевые машины со значением меньше 100
df_cheap[df_cheap < 100]

Chrysler            97
Jeep                94
GAZ                 82
Daewoo              79
Mini                76
Infiniti            72
Dodge               66
Lincoln             57
Moskvich / Izh      51
Alfa Romeo          46
Smart               42
UAZ                 41
Ssangyong           34
Daihatsu            31
Cadillac            27
Alta marca          27
Great Wall          21
Zaz                 20
Chery               19
Lancia              18
Byd                 17
Saab                15
Lada                14
Acura               14
Lifan               12
Isuzu               11
LuAZ                10
Ravon                5
Buick                5
Changan              4
Datsun               3
Geely                3
Haima                3
Scion                3
Haval                3
BAIC                 3
Jac                  3
Pontiac              2
Faw                  2
Mercedes-Maybach     2
ARO                  2
Groz                 1
Abarth               1
Zotye      

In [442]:
# Выделим дорогие машины (цена выше средней)
df_car2 = df_car[(df_car["Price(euro)"] >= mean_price)]
mask2 = df_car["Price(euro)"] >= mean_price

In [443]:
df2 = df["Make"].value_counts()
df_exp = df2[mask2]
df_exp

Land Rover      313
Porsche         204
Jaguar           71
Tesla            28
Brilliance       16
GMC              10
Hummer            5
Bentley           5
Maserati          4
Ferrari           1
McLaren           1
Aston Martin      1
Xpeng             1
Name: Make, dtype: int64

In [444]:
df_exp.describe()

count     13.000000
mean      50.769231
std       96.750671
min        1.000000
25%        1.000000
50%        5.000000
75%       28.000000
max      313.000000
Name: Make, dtype: float64

In [445]:
# Выделим число редких дорогих машин со значением меньше 100
df_exp[df_exp < 100]

Jaguar          71
Tesla           28
Brilliance      16
GMC             10
Hummer           5
Bentley          5
Maserati         4
Ferrari          1
McLaren          1
Aston Martin     1
Xpeng            1
Name: Make, dtype: int64

In [446]:
# Заменяем исходные значения на редкие дорогие rare_exp и редкие дешевые rare_cheap
rare_cheap =  df_cheap[(df_cheap < 100)]
rare_exp = df_exp[(df_exp < 100)]
df['Make'] = df['Make'].replace(rare_cheap.index.values, 'Rare_cheap_cars')
df['Make'] = df['Make'].replace(rare_exp.index.values, 'Rare_exp_cars')
df.Make.value_counts()

Volkswagen         3953
Mercedes           3501
Toyota             3427
BMW                2998
Renault            2685
Opel               2220
Skoda              1892
Audi               1786
Ford               1527
Dacia              1524
Nissan             1464
Hyundai            1341
Rare_cheap_cars    1142
Honda              1029
KIA                 750
Mitsubishi          729
Vaz                 654
Lexus               651
Volvo               636
Mazda               539
Peugeot             469
Chevrolet           365
Citroen             337
Land Rover          313
Seat                286
Fiat                237
Porsche             204
Suzuki              176
Subaru              171
Rare_exp_cars       143
Rover               115
Name: Make, dtype: int64

Таким образом, были заменены редкие дорогие и редкие дешевые автомобили на Rare_exp_cars и Rare_cheap_cars соответственно. Из полученных данных видно, что число редких дорогих машин (143) намного меньше числа редких дешевых машин (1142), что является логичным.

### 4.4 Категории «с небольшим пробегом», «со средним пробегом» и «с большим пробегом»

In [447]:
# Создадим столбец km_year
df['Age'] = 2022 - df.Year
df['km_year'] = df.Distance/df.Age

In [448]:
df.km_year.describe()

count    3.726400e+04
mean     3.112490e+04
std      5.757222e+05
min      0.000000e+00
25%      8.314375e+03
50%      1.366667e+04
75%      1.850000e+04
max      1.000000e+08
Name: km_year, dtype: float64

In [449]:
# Рассчитаем 0.75-квантиль и 0.25-квантиль
q75 = int(np.quantile(df['km_year'], 0.75))
q25 = int(np.quantile(df['km_year'], 0.25))
q25, q75

(8314, 18500)

In [450]:
# Создадим новый столбец в который в будущем запишем категории пробега
df["km_year_new"] = df["km_year"]
df.head(10)

,Make,Model,Year,Style,Distance,Engine_capacity(cm3),Fuel_type,Transmission,Price(euro),Age,km_year,km_year_new
0,Toyota,Prius,2011,Hatchback,195000,1800,Hybrid,Automatic,7750,11,17727.272727,17727.272727
1,Renault,Grand Scenic,2014,Universal,135000,1500,Diesel,Manual,8550,8,16875.000000,16875.000000
2,Volkswagen,Golf,1998,Hatchback,1,1400,Petrol,Manual,2200,24,0.041667,0.041667
3,Renault,Laguna,2012,Universal,110000,1500,Diesel,Manual,6550,10,11000.000000,11000.000000
4,Opel,Astra,2006,Universal,200000,1600,Metan/Propan,Manual,4100,16,12500.000000,12500.000000
5,Mercedes,Vito,2000,Microvan,300000,2200,Diesel,Manual,3490,22,13636.363636,13636.363636
6,Volkswagen,Passat,2010,Universal,290000,1390,Petrol,Manual,5600,12,24166.666667,24166.666667
7,Volkswagen,Touareg,2010,SUV,220000,2400,Diesel,Automatic,17000,12,18333.333333,18333.333333
8,Mercedes,Series (W124),1991,Sedan,240000,2300,Petrol,Manual,2850,31,7741.935484,7741.935484
9,BMW,5 Series,2004,Sedan,325000,300,Diesel,Manual,3400,18,18055.555556,18055.555556


In [451]:
# Сделаем соответствующую замену на небольшой, средний и большой пробеги
df["km_year_new"] = df["km_year_new"].apply(lambda x: (int(x) >= q75 and "Большой пробег") or (int(x) < q25 and "Небольшой пробег") or "Средний пробег") 

In [452]:
df.head(10)

,Make,Model,Year,Style,Distance,Engine_capacity(cm3),Fuel_type,Transmission,Price(euro),Age,km_year,km_year_new
0,Toyota,Prius,2011,Hatchback,195000,1800,Hybrid,Automatic,7750,11,17727.272727,Средний пробег
1,Renault,Grand Scenic,2014,Universal,135000,1500,Diesel,Manual,8550,8,16875.000000,Средний пробег
2,Volkswagen,Golf,1998,Hatchback,1,1400,Petrol,Manual,2200,24,0.041667,Небольшой пробег
3,Renault,Laguna,2012,Universal,110000,1500,Diesel,Manual,6550,10,11000.000000,Средний пробег
4,Opel,Astra,2006,Universal,200000,1600,Metan/Propan,Manual,4100,16,12500.000000,Средний пробег
5,Mercedes,Vito,2000,Microvan,300000,2200,Diesel,Manual,3490,22,13636.363636,Средний пробег
6,Volkswagen,Passat,2010,Universal,290000,1390,Petrol,Manual,5600,12,24166.666667,Большой пробег
7,Volkswagen,Touareg,2010,SUV,220000,2400,Diesel,Automatic,17000,12,18333.333333,Средний пробег
8,Mercedes,Series (W124),1991,Sedan,240000,2300,Petrol,Manual,2850,31,7741.935484,Небольшой пробег
9,BMW,5 Series,2004,Sedan,325000,300,Diesel,Manual,3400,18,18055.555556,Средний пробег


In [457]:
df["km_year_new"].value_counts()

Средний пробег      18605
Большой пробег       9343
Небольшой пробег     9316
Name: km_year_new, dtype: int64

In [454]:
# Далее для удобной работы можно сделать One Hot Encoding
df_one = pd.get_dummies(df, columns=['km_year_new'])
df_one.head(10)

,Make,Model,Year,Style,Distance,Engine_capacity(cm3),Fuel_type,Transmission,Price(euro),Age,km_year,km_year_new_Большой пробег,km_year_new_Небольшой пробег,km_year_new_Средний пробег
0,Toyota,Prius,2011,Hatchback,195000,1800,Hybrid,Automatic,7750,11,17727.272727,0,0,1
1,Renault,Grand Scenic,2014,Universal,135000,1500,Diesel,Manual,8550,8,16875.000000,0,0,1
2,Volkswagen,Golf,1998,Hatchback,1,1400,Petrol,Manual,2200,24,0.041667,0,1,0
3,Renault,Laguna,2012,Universal,110000,1500,Diesel,Manual,6550,10,11000.000000,0,0,1
4,Opel,Astra,2006,Universal,200000,1600,Metan/Propan,Manual,4100,16,12500.000000,0,0,1
5,Mercedes,Vito,2000,Microvan,300000,2200,Diesel,Manual,3490,22,13636.363636,0,0,1
6,Volkswagen,Passat,2010,Universal,290000,1390,Petrol,Manual,5600,12,24166.666667,1,0,0
7,Volkswagen,Touareg,2010,SUV,220000,2400,Diesel,Automatic,17000,12,18333.333333,0,0,1
8,Mercedes,Series (W124),1991,Sedan,240000,2300,Petrol,Manual,2850,31,7741.935484,0,1,0
9,BMW,5 Series,2004,Sedan,325000,300,Diesel,Manual,3400,18,18055.555556,0,0,1


Из полученных данных можно сделать вывод, что в большинстве случаев автомобили имеют средний пробег. В нашем случае средний пробег имеют 18605 автомобилей, в то время как большой - 9343 и маленький - 9316 автомобилей